In [78]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import pandas as pd

In [119]:
# get Senate map Soup opbject
on_lower = 'house'
URL = f'https://www.sos.state.nm.us/voting-and-elections/data-and-maps/{on_lower}-district-maps/'
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [120]:
tables = soup.find('table')
dict_list = []
for i in tables.tbody.findAll('tr'):
    dis_text = i.find_next('td').text
    lnk_text = i.find_all('td')[1].find_next('a')['href']
    if (re.search('[0-9]+', dis_text) is not None): 
        dict_list.append({f'{on_lower}_district':re.findall('[0-9]+', dis_text)[0], 
                       'Map Link':lnk_text})
df_link = pd.DataFrame(dict_list)
df_link.head()
#df_link.dtypes

,house_district,Map Link
0,1,https://api.realfile.rtsclients.com/PublicFile...
1,2,https://api.realfile.rtsclients.com/PublicFile...
2,3,https://api.realfile.rtsclients.com/PublicFile...
3,4,https://api.realfile.rtsclients.com/PublicFile...
4,5,https://api.realfile.rtsclients.com/PublicFile...


In [127]:
# join with legislator file
ld = pd.read_csv(f'./nm_{on_lower}_legislator_info.csv').drop(['Map Link'], axis = 1)
print(ld.head())
print(ld.dtypes)
#ld['District'] = ld['District'].astype('str')
df_link[f'{on_lower}_district'] = df_link[f'{on_lower}_district'].astype('int64')
ld.merge(df_link, how = 'left', left_on = 'District', right_on = f'{on_lower}_district').drop([f'{on_lower}_district'], axis = 1)

         Legislator Party Office  District             County  \
0  Eliseo Lee Alcon     D  House         6  Cibola & McKinley   
1   Anthony Allison     D  House         4           San Juan   

                     Service        Occupation  \
0  Representative since 2009  Disabled veteran   
1  Representative since 2019               NaN   

                             Address Capitol Room  Office Phone  \
0          Box 2134, Milan, NM 87021         314A           NaN   
1  35 Road 6785, Fruitland, NM 87416         413C           NaN   

       Home Phone                        Email  
0  (505) 285-6387     eliseo.alcon@nmlegis.gov  
1  (505) 787-8494  anthony.allison@nmlegis.gov  
Legislator       object
Party            object
Office           object
District          int64
County           object
Service          object
Occupation       object
Address          object
Capitol Room     object
Office Phone    float64
Home Phone       object
Email            object
dtype: object


,Legislator,Party,Office,District,County,Service,Occupation,Address,Capitol Room,Office Phone,Home Phone,Email,Map Link
0,Eliseo Lee Alcon,D,House,6,Cibola & McKinley,Representative since 2009,Disabled veteran,"Box 2134, Milan, NM 87021",314A,NaN,(505) 285-6387,eliseo.alcon@nmlegis.gov,https://api.realfile.rtsclients.com/PublicFile...
1,Anthony Allison,D,House,4,San Juan,Representative since 2019,NaN,"35 Road 6785, Fruitland, NM 87416",413C,NaN,(505) 787-8494,anthony.allison@nmlegis.gov,https://api.realfile.rtsclients.com/PublicFile...
